In [28]:
import boto3
from pprint import pprint
from pprint import pprint
ec2_cli = boto3.client('ec2')
list_of_volid=[]
snapids=[]
'''
for each_vol in ec2_cli.describe_volumes(Filters=[{'Name':'tag:Prod','Values':['Devops']}])['Volumes']:
	list_of_volid.append(each_vol['VolumeId'])
	print(each_vol)
print("list of volume ids are", list_of_volid)
'''
paginator=ec2_cli.get_paginator('describe_volumes')
for each_page in paginator.paginate(Filters=[{'Name':'tag:Prod','Values':['Devops']}]):
    for each_vol in each_page['Volumes']:
        #pprint(each_vol['VolumeId'])
        list_of_volid.append(each_vol['VolumeId'])
print("The List of volume id's are: ", list_of_volid)
for each_volid in list_of_volid:
    print("Taking snap of {}".format(each_volid))
    resp=ec2_cli.create_snapshot(
        VolumeId=each_volid,
        TagSpecifications=[
            {
                'ResourceType':'snapshot',
                'Tags':[
                    {
                        'Key':'snap_shot',
                        'Value':'90'
                    }
                ]
            }
        ],
              
                              )
    snapids.append(resp['SnapshotId'])
    waiter = ec2_cli.get_waiter('snapshot_completed')
    waiter.wait(SnapshotIds=snapids)
    print("The Snap id's are {}".format(snapids))


The List of volume id's are:  ['vol-0b5e5ebd2ef73b46c', 'vol-054b32b2b6ef45872']
Taking snap of vol-0b5e5ebd2ef73b46c
The Snap id's are ['snap-09d0ce56d6c94dd96']
Taking snap of vol-054b32b2b6ef45872
The Snap id's are ['snap-09d0ce56d6c94dd96', 'snap-0ce81f5b8db601ac6']


## The filters we have to take is for volumes not for ec2 